# Installs and Imports

In [1]:
# !git clone --recursive https://github.com/Microsoft/LightGBM 
# %cd LightGBM 
# !mkdir build
# %cd build
# !cmake -DUSE_GPU=1 ..
# !make -j4
# !pip uninstall lightgbm
# !cd ../python-package/ && python setup.py install
# !pip install xgboost --upgrade
!pip install sklego
!pip install lightgbm --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 224 kB 8.6 MB/s 
     |████████████████████████████████| 88 kB 6.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.3 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=37c202095b7a920ac25a68fea92425a3c791a9b2b4d44ae075676b9c11f4ffab
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=d3b1f2b2e7c04a4cf83257f931e34169a56091b6fd0c5b9f483f1d132dee8c4d
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent


In [ ]:
%cd ../../

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import gc, os
import random
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PowerTransformer
from sklearn.mixture import BayesianGaussianMixture
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import lightgbm as lgb

import xgboost as xgb

from sklego.mixture import BayesianGMMClassifier

from drive.MyDrive.Kaggle.Clustering_072022.src import functions

import warnings # suppress warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Configuration

In [7]:
class CFG:
    path = 'drive/MyDrive/Kaggle/Clustering_072022/'
    alpha = 0.05
    n_components = 7
    max_iter = 500
    n_init=3
    tol = 0.0001
    seed = 0
    threshold = 0.75
    # prob_coef = 1.0
    # prob1_coef = 0.9
    # prob2_coef = 0.85
    # prob3_coef = 0.85
    n_folds = 3

# params_lgb = {'learning_rate': 0.07,
#               'objective': 'multiclass',
#               'boosting': 'gbdt',
#               'verbosity': -1,
#               'n_jobs': -1,
#               'num_classes':CFG.n_components} 

functions.set_seed(CFG.seed)

# Data Importation and Transformation

In [8]:
data = pd.read_csv(CFG.path + 'src/data.csv', index_col='id')
cols = data.columns.tolist()

In [9]:
best_cols = []

for col in cols:
    _, p_value = stats.shapiro(data[col])
    if p_value <= CFG.alpha:
        best_cols.append(col)
all_cols, cols = cols, best_cols

In [10]:
def PT(data, cols):
    PT = PowerTransformer()
    data[cols] = PT.fit_transform(data[cols])
    return data

def transform(df):
    return (
        df[cols]
        .astype({
            'f_07': 'int8',
            'f_08': 'int8',
            'f_09': 'int8',
            'f_10': 'int8',
            'f_11': 'int8',
            'f_12': 'int8',
            'f_13': 'int8',
        })
        .pipe(PT, cols)
        # .select_dtypes(['int8', 'float64'])
        )
    
df = transform(data)

In [11]:
df

,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_22,f_23,f_24,f_25,f_26,f_27,f_28
id,,,,,,,,,,,,,,
0,-0.98,1.38,1.04,-0.57,1.14,-0.08,0.09,-0.71,0.91,-0.68,0.77,0.96,1.04,0.69
1,-0.98,-0.88,-0.18,-0.84,-0.42,1.73,0.74,-0.54,0.45,1.03,-0.12,-0.55,0.37,-1.64
2,0.02,1.02,-0.39,0.12,0.30,-0.93,1.12,2.20,0.09,-1.52,-0.57,0.98,-0.93,-2.30
3,0.29,-1.21,0.92,0.12,0.30,-0.08,-0.43,0.73,-1.22,0.83,-1.17,-0.40,-0.10,0.33
4,0.76,0.19,-0.39,-1.14,-1.95,1.27,1.12,0.23,-1.48,0.85,-0.61,1.16,-0.37,-1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97995,0.02,0.19,0.19,-0.57,-0.17,-0.08,0.32,0.95,0.41,1.28,-0.08,-0.99,0.17,0.46
97996,-0.60,0.62,-1.24,-0.57,-1.26,0.94,1.30,1.16,-0.43,0.27,0.78,0.14,0.69,0.64
97997,0.76,0.62,0.51,-0.09,-0.68,-0.61,-0.74,-1.00,1.45,0.87,0.18,1.54,1.18,0.58


# Create Probability Array using BGMM

In [12]:
best_preds = pd.read_csv(CFG.path + 'submissions/fine_art.csv', index_col='Id')

bgmmc = BayesianGMMClassifier(n_components=CFG.n_components, max_iter=CFG.max_iter, n_init=CFG.n_init, tol=CFG.tol)
bgmmc.fit(df, best_preds['Predicted'])
probas = bgmmc.predict_proba(df)

In [13]:
df['predictions'] = best_preds['Predicted']
df['predict_proba'] = 0

for i in range(CFG.n_components):
    df[f'predict_proba_{i}'] = probas[:, i]
    df.loc[df['predictions'] == i, 'predict_proba'] = df[f'predict_proba_{i}']

# Select for High Confidence Data

In [14]:
feature_pct = 0
bgm_high_conf_data = np.array([])

for i in range(CFG.n_components):
    median_prob = df[df['predictions'] == i]['predict_proba'].median()
    idx = df[(df['predictions'] == i) & (df['predict_proba'] > CFG.threshold)].index
    feature_pct = len(idx) / len(df[df['predictions'] == i])

    print(f'Cluster: {i}, Median Probability: {median_prob : .3f}, Feature Percent: {feature_pct : .2f}')
    bgm_high_conf_data = np.concatenate((bgm_high_conf_data, idx))

Cluster: 0, Median Probability:  0.996, Feature Percent:  0.90
Cluster: 1, Median Probability:  0.998, Feature Percent:  0.91
Cluster: 2, Median Probability:  0.992, Feature Percent:  0.88
Cluster: 3, Median Probability:  0.999, Feature Percent:  0.95
Cluster: 4, Median Probability:  0.996, Feature Percent:  0.92
Cluster: 5, Median Probability:  0.992, Feature Percent:  0.89
Cluster: 6, Median Probability:  0.985, Feature Percent:  0.86


# LGBM

In [15]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

X_train = df.loc[bgm_high_conf_data][cols]
y_train = df.loc[bgm_high_conf_data]['predictions']

params_lgb = {'learning_rate': 0.003,
              'objective': 'multiclass',
              'boosting': 'gbdt',
              'verbosity': -1,
              'n_jobs': -1,
              'num_classes':CFG.n_components,
              'num_leaves': 128
              } 

model_list = []
lgbm_predictions_prob = 0
skf = StratifiedKFold(CFG.n_folds)
lgbm_y_probas = np.array([])
lgbm_y_preds = 0

for fold, (trn_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    print('====='*5, f'FOLD: {fold}', '====='*5)

    train_dataset = lgb.Dataset(X_train.iloc[trn_idx], y_train.iloc[trn_idx], feature_name=cols)
    val_dataset = lgb.Dataset(X_train.iloc[val_idx], y_train.iloc[val_idx], feature_name=cols)

    model = lgb.train(
        params=params_lgb,
        train_set=train_dataset,
        valid_sets=val_dataset,
        num_boost_round=10000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=300, verbose=True),
            lgb.log_evaluation(period=200)
        ])
    
    model_list.append(model)

    y_pproba = model.predict(X_train.iloc[val_idx])
    y_pred = np.argmax(y_pproba, axis=1)

    score = balanced_accuracy_score(y_train.iloc[val_idx], y_pred)
    auc = roc_auc_score(y_train.iloc[val_idx], y_pproba, average='weighted', multi_class='ovo')

    lgbm_predictions_prob += model.predict(df[cols]) / CFG.n_folds

    print(f'LGBM AUC : {score:.3f} | ACC : {auc:.1%}\n')
    print('.' * 10)
    print('')

========================= FOLD: 0 =========================
Training until validation scores don't improve for 300 rounds
[200]	valid_0's multi_logloss: 0.972417
[400]	valid_0's multi_logloss: 0.653722
[600]	valid_0's multi_logloss: 0.500361
[800]	valid_0's multi_logloss: 0.409841
[1000]	valid_0's multi_logloss: 0.353665
[1200]	valid_0's multi_logloss: 0.316972
[1400]	valid_0's multi_logloss: 0.291736
[1600]	valid_0's multi_logloss: 0.273567
[1800]	valid_0's multi_logloss: 0.260151
[2000]	valid_0's multi_logloss: 0.250648
[2200]	valid_0's multi_logloss: 0.243398
[2400]	valid_0's multi_logloss: 0.237787
[2600]	valid_0's multi_logloss: 0.233678
[2800]	valid_0's multi_logloss: 0.230505
[3000]	valid_0's multi_logloss: 0.227647
[3200]	valid_0's multi_logloss: 0.225524
[3400]	valid_0's multi_logloss: 0.223732
[3600]	valid_0's multi_logloss: 0.222601
[3800]	valid_0's multi_logloss: 0.221677
[4000]	valid_0's multi_logloss: 0.221026
[4200]	valid_0's multi_logloss: 0.220642
[4400]	valid_0's mult

# Encorporating LGBM Data

In [18]:
df['lgbm_preds'] = np.argmax(lgbm_predictions_prob, axis=1)
df['lgbm_predict_proba'] = 0

for i in range(CFG.n_components):
    df[f'lgbm_predict_proba_{i}'] = lgbm_predictions_prob[:, i]
    df.loc[df['lgbm_preds'] == i, 'lgbm_predict_proba'] = df[f'lgbm_predict_proba_{i}']

# Selecting for High Confidence Data

In [23]:
feature_pct = 0
high_conf_data = np.array([])

for i in range(CFG.n_components):
    median_prob = df[df['lgbm_preds'] == i]['lgbm_predict_proba'].median()
    idx = df[(df['lgbm_preds'] == i) & (df['lgbm_predict_proba'] > CFG.threshold)].index
    feature_pct = len(idx) / len(df[df['lgbm_preds'] == i])

    print(f'Cluster: {i}, Median Probability: {median_prob : .3f}, Feature Percent: {feature_pct : .2f}')
    high_conf_data = np.concatenate((high_conf_data, idx))

Cluster: 0, Median Probability:  0.996, Feature Percent:  0.92
Cluster: 1, Median Probability:  0.994, Feature Percent:  0.89
Cluster: 2, Median Probability:  0.990, Feature Percent:  0.89
Cluster: 3, Median Probability:  1.000, Feature Percent:  0.96
Cluster: 4, Median Probability:  0.992, Feature Percent:  0.90
Cluster: 5, Median Probability:  0.992, Feature Percent:  0.90
Cluster: 6, Median Probability:  0.953, Feature Percent:  0.79


In [1]:
lgbm_cols = cols + ['lgbm_preds']


NameError: ignored

In [42]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

X_train = df.loc[high_conf_data][cols]
y_train = df.loc[high_conf_data]['lgbm_preds']

params_lgb = {'learning_rate': 0.07,
              'objective': 'multiclass',
              'boosting': 'gbdt',
              'verbosity': -1,
              'n_jobs': -1,
              'num_classes':CFG.n_components,
              'n_estimators': 100,
              'num_leaves': 31} 

model_list = []
lgbm_predictions_prob = 0
skf = StratifiedKFold(CFG.n_folds)
lgbm_y_probas = np.array([])
lgbm_y_preds = 0

for fold, (trn_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    print('====='*5, f'FOLD: {fold}', '====='*5)

    train_dataset = lgb.Dataset(X_train.iloc[trn_idx], y_train.iloc[trn_idx], feature_name=cols)
    val_dataset = lgb.Dataset(X_train.iloc[val_idx], y_train.iloc[val_idx], feature_name=cols)

    model = lgb.train(
        params=params_lgb,
        train_set=train_dataset,
        valid_sets=val_dataset,
        num_boost_round=100000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=300, verbose=True),
            lgb.log_evaluation(period=200)
        ])
    
    model_list.append(model)

    y_pproba = model.predict(X_train.iloc[val_idx])
    y_pred = np.argmax(y_pproba, axis=1)

    score = balanced_accuracy_score(y_train.iloc[val_idx], y_pred)
    auc = roc_auc_score(y_train.iloc[val_idx], y_pproba, average='weighted', multi_class='ovo')

    lgbm_predictions_prob += model.predict(df[cols]) / CFG.n_folds

    print(f'LGBM AUC : {score:.3f} | ACC : {auc:.1%}\n')
    print('.' * 10)
    print('')

========================= FOLD: 0 =========================
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.166582
LGBM AUC : 0.947 | ACC : 99.8%

..........

========================= FOLD: 1 =========================
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.166636
LGBM AUC : 0.948 | ACC : 99.8%

..........

========================= FOLD: 2 =========================
Training until validation scores don't improve for 300 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.168476
LGBM AUC : 0.947 | ACC : 99.8%

..........



In [32]:
submission = pd.read_csv(CFG.path + 'submissions/sample_submission.csv', index_col='Id')
submission['Predicted'] = bgm_preds
submission.to_csv(CFG.path + 'submissions/iterative_clustering#1.csv', index='Id')

In [39]:
df

,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_22,f_23,f_24,f_25,f_26,f_27,f_28,predictions,predict_proba,predict_proba_0,predict_proba_1,predict_proba_2,predict_proba_3,predict_proba_4,predict_proba_5,predict_proba_6,lgbm_preds,lgbm_predict_proba,lgbm_predict_proba_0,lgbm_predict_proba_1,lgbm_predict_proba_2,lgbm_predict_proba_3,lgbm_predict_proba_4,lgbm_predict_proba_5,lgbm_predict_proba_6
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.98,1.38,1.04,-0.57,1.14,-0.08,0.09,-0.71,0.91,-0.68,0.77,0.96,1.04,0.69,0,0.99,0.99,0.00,0.00,0.00,0.00,0.00,0.00,0,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
1,-0.98,-0.88,-0.18,-0.84,-0.42,1.73,0.74,-0.54,0.45,1.03,-0.12,-0.55,0.37,-1.64,6,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,6,0.99,0.00,0.01,0.00,0.00,0.00,0.00,0.99
2,0.02,1.02,-0.39,0.12,0.30,-0.93,1.12,2.20,0.09,-1.52,-0.57,0.98,-0.93,-2.30,5,0.95,0.05,0.00,0.00,0.00,0.00,0.95,0.00,5,0.74,0.26,0.00,0.00,0.00,0.00,0.74,0.00
3,0.29,-1.21,0.92,0.12,0.30,-0.08,-0.43,0.73,-1.22,0.83,-1.17,-0.40,-0.10,0.33,3,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,3,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
4,0.76,0.19,-0.39,-1.14,-1.95,1.27,1.12,0.23,-1.48,0.85,-0.61,1.16,-0.37,-1.16,2,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,2,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97995,0.02,0.19,0.19,-0.57,-0.17,-0.08,0.32,0.95,0.41,1.28,-0.08,-0.99,0.17,0.46,6,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,6,0.93,0.00,0.00,0.05,0.00,0.02,0.00,0.93
97996,-0.60,0.62,-1.24,-0.57,-1.26,0.94,1.30,1.16,-0.43,0.27,0.78,0.14,0.69,0.64,4,0.93,0.00,0.00,0.06,0.00,0.93,0.00,0.00,4,0.97,0.00,0.00,0.03,0.00,0.97,0.00,0.00
97997,0.76,0.62,0.51,-0.09,-0.68,-0.61,-0.74,-1.00,1.45,0.87,0.18,1.54,1.18,0.58,5,0.93,0.00,0.00,0.07,0.00,0.00,0.93,0.00,5,0.97,0.00,0.00,0.02,0.00,0.00,0.97,0.01


In [40]:
cols

['f_07',
 'f_08',
 'f_09',
 'f_10',
 'f_11',
 'f_12',
 'f_13',
 'f_22',
 'f_23',
 'f_24',
 'f_25',
 'f_26',
 'f_27',
 'f_28']

In [41]:
high_conf_data

array([0.0000e+00, 7.0000e+00, 8.0000e+00, ..., 9.7987e+04, 9.7994e+04,
       9.7995e+04])